In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_40.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 54 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(54): SRR7817614 SRR1636589 ... SRR1636673 SRR7817631
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 54
                   SRR7817614 SRR1636589 SRR7817660 SRR7817635 SRR3593524
ENSMUSG00000000001  15.766115  16.206252  15.903869  15.643068  15.187207
ENSMUSG00000000028   8.192656   8.033180   8.178474   8.227609   8.052549
ENSMUSG00000000049  20.428537  20.488350  20.107555  20.364601  20.511828
ENSMUSG00000000058   9.074042   9.461244   8.880225   9.012768   8.884885
ENSMUSG00000000085  11.080172  10.271753  10.981584  10.914202  11.019693
ENSMUSG00000000088  16.883814  15.965222  16.253726  16.296999  15.965910
                   SRR7817651 SRR7817675 SRR7817687 SRR7817676 SRR7817673
ENSMUSG00000000001  15.864994  15.834032  15.812683  15.842664  15.639946
ENSMUSG00000000028   8.493369   8.332907   8.260709   8.283368   8.416844
ENSMUSG00000000049  20.136944  19.971094  20.233782  19.979789  20.035165
ENSMUSG00000000058   8.889686   9.320651   8.763093   9.459410   8.965462
ENSMUSG00000000085  10.707461  10.312766  10.247609  10.266789  10.773110
ENSMUSG00000000088  16.386229  

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_40.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[51] 30 30 30 30


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_40.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[49] "ENSMUSG00000095170" "ENSMUSG00000095210" "ENSMUSG00000095285"
[52] "ENSMUSG00000095429" "ENSMUSG00000095497" "ENSMUSG00000095571"
[55] "ENSMUSG00000095612" "ENSMUSG00000095737" "ENSMUSG00000096336"
[58] "ENSMUSG00000096407" "ENSMUSG00000096580" "ENSMUSG00000102301"
[61] "ENSMUSG00000102364" "ENSMUSG00000103033" "ENSMUSG00000103939"
[64] "ENSMUSG00000104452" "ENSMUSG00000105606" "ENSMUSG00000105906"
[67] "ENSMUSG00000106403"

$`R-MMU-173631`
 [1] "ENSMUSG00000036887" "ENSMUSG00000036896" "ENSMUSG00000036905"
 [4] "ENSMUSG00000055172" "ENSMUSG00000075370" "ENSMUSG00000076500"
 [7] "ENSMUSG00000076501" "ENSMUSG00000076505" "ENSMUSG00000076514"
[10] "ENSMUSG00000076518" "ENSMUSG00000076522" "ENSMUSG00000076523"
[13] "ENSMUSG00000076525" "ENSMUSG00000076535" "ENSMUSG00000076572"
[16] "ENSMUSG00000076573" "ENSMUSG00000076586" "ENSMUSG00000076612"
[19] "ENSMUSG00000076614" "ENSMUSG00000076615" "ENSMUSG00000076633"
[22] "ENSMUSG00000076652" "ENSMUSG00000076653" "ENSMUSG00000076668"
[25] 

[1] "ENSMUSG00000038224" "ENSMUSG00000059481"

$`R-MMU-202726`
[1] "ENSMUSG00000038235"

$`R-MMU-9717432`
[1] "ENSMUSG00000038260"

$`R-MMU-434989`
[1] "ENSMUSG00000038264" "ENSMUSG00000074785"

$`R-MMU-9649963`
[1] "ENSMUSG00000038276"

$`R-MMU-6784959`
[1] "ENSMUSG00000038286"

$`R-MMU-8985270`
[1] "ENSMUSG00000038331" "ENSMUSG00000039153"

$`R-MMU-1222685`
[1] "ENSMUSG00000038357"

$`R-MMU-6801687`
[1] "ENSMUSG00000038357" "ENSMUSG00000057729"

$`R-MMU-6787632`
[1] "ENSMUSG00000038372"

$`R-MMU-8953111`
[1] "ENSMUSG00000038482" "ENSMUSG00000057469"

$`R-MMU-199626`
[1] "ENSMUSG00000038496" "ENSMUSG00000040918"

$`R-MMU-9705794`
[1] "ENSMUSG00000038541"

$`R-MMU-374673`
[1] "ENSMUSG00000038546"

$`R-MMU-209765`
[1] "ENSMUSG00000038567"

$`R-MMU-211950`
[1] "ENSMUSG00000038567"

$`R-MMU-192033`
[1] "ENSMUSG00000038641"

$`R-MMU-192067`
[1] "ENSMUSG00000038641"

$`R-MMU-193746`
[1] "ENSMUSG00000038641"

$`R-MMU-193755`
[1] "ENSMUSG00000038641"

$`R-MMU-193821`
[1] "ENSMUSG00000038641"


[1] "ENSMUSG00000040693"

$`R-MMU-211874`
[1] "ENSMUSG00000040703"

$`R-MMU-350604`
[1] "ENSMUSG00000040706"

$`R-MMU-373717`
[1] "ENSMUSG00000040760" "ENSMUSG00000060534"

$`R-MMU-1482961`
[1] "ENSMUSG00000040774"

$`R-MMU-1482962`
[1] "ENSMUSG00000040774" "ENSMUSG00000075703"

$`R-MMU-264418`
[1] "ENSMUSG00000040782"

$`R-MMU-264435`
[1] "ENSMUSG00000040782" "ENSMUSG00000059552"

$`R-MMU-4167511`
[1] "ENSMUSG00000040820" "ENSMUSG00000042010"

$`R-MMU-5635102`
[1] "ENSMUSG00000040836"

$`R-MMU-3209159`
[1] "ENSMUSG00000040857"

$`R-MMU-1471338`
[1] "ENSMUSG00000040899" "ENSMUSG00000044748" "ENSMUSG00000046354"
[4] "ENSMUSG00000059230"

$`R-MMU-1299338`
[1] "ENSMUSG00000040901"

$`R-MMU-5694583`
[1] "ENSMUSG00000040997"

$`R-NUL-9830947`
[1] "ENSMUSG00000040998"

$`R-MMU-5423117`
 [1] "ENSMUSG00000041229" "ENSMUSG00000060093" "ENSMUSG00000060639"
 [4] "ENSMUSG00000060678" "ENSMUSG00000060981" "ENSMUSG00000061482"
 [7] "ENSMUSG00000064288" "ENSMUSG00000067455" "ENSMUSG00000069266"
[10] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




$`R-MMU-159179`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-159194`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756134`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756156`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756183`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9632038`
[1] "ENSMUSG00000089960"

$`R-MMU-9632039`
[1] "ENSMUSG00000089960"

$`R-MMU-5695989`
[1] "ENSMUSG00000090150"

$`R-MMU-209821`
[1] "ENSMUSG00000093806"

$`R-MMU-191323`
[1] "ENSMUSG00000093930"

$`R-MMU-159790`
[1] "ENSMUSG00000096145"

$`R-MMU-9026967`
[1] "ENSMUSG00000096145"

$`R-MMU-5619415`
[1] "ENSMUSG00000101174"

$`R-MMU-6807826`
[1] "ENSMUSG00000101959"

$`R-MMU-1655879`
[1] "ENSMUSG00000103711"

$`R-MMU-444419`
[1] "ENSMUSG00000104445"

$`R-MMU-446278`
[1] "ENSMUSG00000104445"

$`R-MMU-6786789`
[1] "ENSMUSG00000110206"

$`R-MMU-8854736`
[1] "ENSMUSG00000110206"

$`R-MMU-9604767`
[1] "ENSMUSG00000110206"

$`R-MMU-9706364`
[1] "ENSMUSG00000110206"

In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

[1] 9825
   [1] "R-MMU-170666"  "R-MMU-170671"  "R-MMU-170674"  "R-MMU-170686" 
   [5] "R-MMU-380073"  "R-MMU-392187"  "R-MMU-392195"  "R-MMU-392202" 
   [9] "R-MMU-749454"  "R-MMU-749456"  "R-MMU-751001"  "R-MMU-8964229"
  [13] "R-MMU-8964242" "R-MMU-8964329" "R-MMU-8982012" "R-MMU-8982019"
  [17] "R-MMU-8982020" "R-MMU-9021596" "R-MMU-9021600" "R-MMU-9021601"
  [21] "R-MMU-176298"  "R-MMU-176318"  "R-MMU-68914"   "R-MMU-68916"  
  [25] "R-MMU-68917"   "R-MMU-68960"   "R-MMU-481009"  "R-MMU-9021068"
  [29] "R-MMU-9021079" "R-MMU-3108203" "R-MMU-3108209" "R-MMU-3927824"
  [33] "R-MMU-4551655" "R-MMU-4551727" "R-MMU-4570463" "R-MMU-4570499"
  [37] "R-MMU-8937989" "R-MMU-163214"  "R-MMU-9709406" "R-MMU-9838627"
  [41] "R-MMU-9839113" "R-MMU-193641"  "R-MMU-193646"  "R-MMU-193650" 
  [45] "R-MMU-193653"  "R-MMU-193656"  "R-MMU-193665"  "R-MMU-193668" 
  [49] "R-MMU-193669"  "R-MMU-193679"  "R-MMU-193682"  "R-MMU-193684" 
  [53] "R-MMU-193694"  "R-MMU-193695"  "R-MMU-193696"  "R-MMU-193700

 [461] "R-MMU-382054"  "R-MMU-382055"  "R-MMU-382056"  "R-MMU-382058" 
 [465] "R-MMU-389083"  "R-MMU-389086"  "R-MMU-481007"  "R-MMU-8864036"
 [469] "R-MMU-8865275" "R-MMU-8865276" "R-MMU-9674093" "R-MMU-8858369"
 [473] "R-MMU-1181149" "R-MMU-1181153" "R-MMU-1225898" "R-MMU-1549526"
 [477] "R-MMU-6803719" "R-MMU-202723"  "R-MMU-2327746" "R-MMU-2328037"
 [481] "R-MMU-2426259" "R-MMU-265424"  "R-MMU-374686"  "R-MMU-4088264"
 [485] "R-MMU-9624017" "R-MMU-975340"  "R-MMU-191790"  "R-MMU-3215385"
 [489] "R-MMU-3215406" "R-MMU-3215426" "R-MMU-3215448" "R-MMU-5205799"
 [493] "R-MMU-5205861" "R-MMU-5218952" "R-MMU-5661117" "R-MMU-418904" 
 [497] "R-MMU-164832"  "R-MMU-164834"  "R-MMU-164840"  "R-MMU-8949580"
 [501] "R-MMU-5626938" "R-MMU-8985343" "R-MMU-9690401" "R-MMU-9690410"
 [505] "R-MMU-9690416" "R-MMU-9690868" "R-MMU-9690898" "R-MMU-9692133"
 [509] "R-MMU-9692136" "R-MMU-9692137" "R-MMU-9692138" "R-MMU-9692139"
 [513] "R-MMU-9692145" "R-MMU-9692156" "R-MMU-9692157" "R-MMU-9692268"
 [517]

 [921] "R-MMU-4332358" "R-MMU-4332359" "R-MMU-4332363" "R-MMU-442749" 
 [925] "R-MMU-445797"  "R-MMU-445813"  "R-MMU-4551451" "R-MMU-4551465"
 [929] "R-MMU-482772"  "R-MMU-5626507" "R-MMU-5626549" "R-MMU-6786205"
 [933] "R-MMU-71541"   "R-MMU-71588"   "R-MMU-9005848" "R-MMU-9005872"
 [937] "R-MMU-9007015" "R-MMU-9619355" "R-MMU-9619376" "R-MMU-9653585"
 [941] "R-MMU-9654521" "R-MMU-9654523" "R-MMU-9654525" "R-MMU-9654533"
 [945] "R-MMU-75885"   "R-MMU-8849345" "R-MMU-427666"  "R-MMU-5336231"
 [949] "R-MMU-5336365" "R-MMU-5336369" "R-MMU-9818467" "R-MMU-9818487"
 [953] "R-MMU-420214"  "R-MMU-420265"  "R-MMU-9733292" "R-MMU-8933256"
 [957] "R-MMU-8933295" "R-MMU-6800198" "R-MMU-6800200" "R-MMU-200318" 
 [961] "R-MMU-2187309" "R-MMU-6810233" "R-MMU-6810234" "R-MMU-6810235"
 [965] "R-MMU-6810238" "R-MMU-6814549" "R-MMU-6814555" "R-MMU-6814559"
 [969] "R-MMU-6814885" "R-MMU-9023840" "R-MMU-9761027" "R-MMU-199032" 
 [973] "R-MMU-265422"  "R-MMU-376419"  "R-MMU-382096"  "R-MMU-5334854"
 [977]

[1381] "R-MMU-6807875" "R-MMU-6807877" "R-MMU-6809010" "R-MMU-6809011"
[1385] "R-MMU-5692770" "R-MMU-5692775" "R-MMU-1011598" "R-MMU-114542" 
[1389] "R-MMU-1433415" "R-MMU-2029451" "R-MMU-2029454" "R-MMU-2029456"
[1393] "R-MMU-2029465" "R-MMU-2029466" "R-MMU-2029467" "R-MMU-2029469"
[1397] "R-MMU-205039"  "R-MMU-205136"  "R-MMU-2130194" "R-MMU-2197690"
[1401] "R-MMU-2424476" "R-MMU-2730840" "R-MMU-2730856" "R-MMU-2730889"
[1405] "R-MMU-376123"  "R-MMU-3858475" "R-MMU-389788"  "R-MMU-3928619"
[1409] "R-MMU-3928620" "R-MMU-3928642" "R-MMU-399930"  "R-MMU-399939" 
[1413] "R-MMU-399952"  "R-MMU-400682"  "R-MMU-418856"  "R-MMU-421142" 
[1417] "R-MMU-421144"  "R-MMU-421147"  "R-MMU-421148"  "R-MMU-442273" 
[1421] "R-MMU-442291"  "R-MMU-442314"  "R-MMU-445064"  "R-MMU-445072" 
[1425] "R-MMU-451347"  "R-MMU-451366"  "R-MMU-5218827" "R-MMU-5218839"
[1429] "R-MMU-5218841" "R-MMU-5218850" "R-MMU-5357445" "R-MMU-5357472"
[1433] "R-MMU-5357483" "R-MMU-5623622" "R-MMU-5623632" "R-MMU-5623667"
[1437]

[1841] "R-MMU-2160932" "R-MMU-2160935" "R-MMU-2169046" "R-MMU-2395364"
[1845] "R-MMU-2731078" "R-MMU-442497"  "R-MMU-8870732" "R-MMU-8878117"
[1849] "R-MMU-8937814" "R-MMU-8951966" "R-MMU-9821366" "R-MMU-445448" 
[1853] "R-MMU-5358336" "R-MMU-5358340" "R-MMU-5358343" "R-MMU-5362386"
[1857] "R-MMU-5362412" "R-MMU-5362427" "R-MMU-5362437" "R-MMU-5362441"
[1861] "R-MMU-5362459" "R-MMU-5362549" "R-MMU-5362551" "R-MMU-5362553"
[1865] "R-MMU-5632649" "R-MMU-5632652" "R-MMU-5632654" "R-MMU-9759844"
[1869] "R-MMU-9815762" "R-NUL-5362792" "R-MMU-109638"  "R-MMU-112381" 
[1873] "R-MMU-113402"  "R-MMU-113407"  "R-MMU-113409"  "R-MMU-6803523"
[1877] "R-MMU-72095"   "R-MMU-75095"   "R-MMU-77095"   "R-MMU-9012315"
[1881] "R-MMU-9012319" "R-MMU-9838035" "R-MMU-9838289" "R-MMU-8853734"
[1885] "R-MMU-8853737" "R-MMU-8853753" "R-MMU-8853755" "R-MMU-8853762"
[1889] "R-MMU-8853774" "R-MMU-8853792" "R-MMU-8853793" "R-MMU-8853801"
[1893] "R-MMU-8854897" "R-MMU-8854899" "R-MMU-8854905" "R-MMU-8854908"
[1897]

...

[5373] "R-MMU-9755013" "R-MMU-448954"  "R-MMU-448960"  "R-MMU-449198" 
[5377] "R-MMU-1454916" "R-MMU-194153"  "R-MMU-9750656" "R-MMU-9753283"
[5381] "R-MMU-8855711" "R-MMU-2066299" "R-MMU-4641147" "R-MMU-5138432"
[5385] "R-MMU-5138441" "R-MMU-508681"  "R-MMU-549505"  "R-MMU-5663147"
[5389] "R-MMU-5663263" "R-MMU-5663294" "R-MMU-5667072" "R-MMU-5667077"
[5393] "R-MMU-74885"   "R-MMU-8942208" "R-MMU-508749"  "R-MMU-549491" 
[5397] "R-MMU-5663158" "R-MMU-210444"  "R-MMU-264615"  "R-MMU-376369" 
[5401] "R-MMU-8867613" "R-MMU-6805122" "R-MMU-6805126" "R-MMU-9023138"
[5405] "R-MMU-73812"   "R-MMU-593680"  "R-MMU-418862"  "R-MMU-418870" 
[5409] "R-MMU-421688"  "R-MMU-421693"  "R-MMU-421694"  "R-MMU-451348" 
[5413] "R-MMU-9011231" "R-MMU-9011252" "R-MMU-2581474" "R-MMU-5696049"
[5417] "R-MMU-75848"   "R-MMU-9733692" "R-MMU-9733738" "R-MMU-73718"  
[5421] "R-MMU-73722"   "R-MMU-5682709" "R-MMU-5682826" "R-MMU-9837337"
[5425] "R-MMU-2534087" "R-MMU-392530"  "R-MMU-5607721" "R-MMU-5607722"
[5429]

[5833] "R-MMU-8948027" "R-MMU-389611"  "R-MMU-3076905" "R-MMU-4167509"
[5837] "R-MMU-5689649" "R-MMU-5690785" "R-MMU-5358494" "R-MMU-5649873"
[5841] "R-MMU-5651828" "R-MMU-422051"  "R-MMU-264622"  "R-MMU-9620456"
[5845] "R-MMU-9621179" "R-MMU-193666"  "R-MMU-205075"  "R-MMU-9008043"
[5849] "R-MMU-9692234" "R-MMU-9692260" "R-MMU-9733784" "R-MMU-9733795"
[5853] "R-MMU-9759850" "R-MMU-9795234" "R-MMU-9823863" "R-MMU-9823870"
[5857] "R-MMU-5661290" "R-MMU-111970"  "R-MMU-198314"  "R-MMU-212552" 
[5861] "R-MMU-450533"  "R-MMU-5218805" "R-MMU-5218813" "R-MMU-5218821"
[5865] "R-MMU-5218823" "R-MMU-5607734" "R-MMU-5607740" "R-MMU-5607746"
[5869] "R-MMU-8934446" "R-MMU-8931974" "R-MMU-163741"  "R-MMU-163751" 
[5873] "R-MMU-71324"   "R-MMU-71335"   "R-MMU-2059926" "R-MMU-190263" 
[5877] "R-MMU-190385"  "R-MMU-6809309" "R-MMU-6809320" "R-MMU-6809325"
[5881] "R-MMU-4551566" "R-MMU-4551572" "R-MMU-5099899" "R-MMU-5140741"
[5885] "R-NUL-3858478" "R-NUL-4551554" "R-NUL-517516"  "R-MMU-5693724"
[5889]

[6293] "R-MMU-9661405" "R-MMU-9843721" "R-MMU-5676917" "R-MMU-176474" 
[6297] "R-MMU-176669"  "R-MMU-212614"  "R-MMU-352119"  "R-NUL-9005747"
[6301] "R-NUL-9005752" "R-MMU-204392"  "R-MMU-204465"  "R-MMU-204485" 
[6305] "R-MMU-204600"  "R-MMU-373867"  "R-MMU-373875"  "R-MMU-375133" 
[6309] "R-MMU-375135"  "R-MMU-9645220" "R-MMU-449115"  "R-MMU-8983294"
[6313] "R-MMU-8983300" "R-MMU-8983307" "R-MMU-8983313" "R-MMU-8983335"
[6317] "R-MMU-8983356" "R-MMU-8983384" "R-MMU-9007126" "R-MMU-9007137"
[6321] "R-MMU-9009867" "R-MMU-158399"  "R-MMU-8852266" "R-MMU-8852481"
[6325] "R-MMU-9021306" "R-MMU-9650473" "R-MMU-373074"  "R-MMU-443978" 
[6329] "R-MMU-5578910" "R-MMU-8936214" "R-MMU-5433074" "R-MMU-9753944"
[6333] "R-MMU-975449"  "R-MMU-1471314" "R-MMU-5621348" "R-MMU-9717206"
[6337] "R-MMU-9728723" "R-MMU-9728748" "R-MMU-70454"   "R-MMU-70481"  
[6341] "R-MMU-162830"  "R-NUL-3876071" "R-MMU-5205652" "R-MMU-2161500"
[6345] "R-MMU-549304"  "R-MMU-561072"  "R-MMU-374896"  "R-MMU-374919" 
[6349]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-9017129" "R-MMU-8874745" "R-MMU-173626"  "R-MMU-173631" 
[8897] "R-MMU-6784224" "R-MMU-8955817" "R-NUL-1458871" "R-MMU-1678921"
[8901] "R-MMU-1678998" "R-MMU-1461993" "R-MMU-1602488" "R-MMU-3214912"
[8905] "R-MMU-5661123" "R-MMU-8849908" "R-MMU-8938887" "R-MMU-374337" 
[8909] "R-MMU-9824464" "R-MMU-9825053" "R-MMU-1605797" "R-MMU-5252072"
[8913] "R-MMU-3296233" "R-NUL-211704"  "R-NUL-211725"  "R-MMU-448480" 
[8917] "R-MMU-8987043" "R-MMU-9009227" "R-MMU-5671763" "R-MMU-1181347"
[8921] "R-NUL-1181349" "R-MMU-8931653" "R-MMU-1678660" "R-MMU-2090085"
[8925] "R-MMU-8932275" "R-NUL-1236944" "R-MMU-9695890" "R-MMU-425983" 
[8929] "R-MMU-426015"  "R-MMU-390886"  "R-MMU-8869206" "R-MMU-196955" 
[8933] "R-MMU-391211"  "R-MMU-435366"  "R-MMU-8938300" "R-MMU-196857" 
[8937]

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

  12.829071  12.798330  12.143682  12.254872
ENSMUSG00000020171  12.317342  12.751254  12.046385  12.345279  12.504319
ENSMUSG00000020175   9.812543   8.670116   9.074513  10.346689  11.528045
ENSMUSG00000020176   9.265524   9.816804   8.677509   8.908185   9.537637
ENSMUSG00000020178   8.731911   7.975723   7.791865   8.096757   8.342717
ENSMUSG00000020180  14.538804  14.157354  13.191797  13.464299  14.075668
ENSMUSG00000020182  16.055177  13.423594  13.168031  14.560353  15.317418
ENSMUSG00000020183   8.512520   9.781069  10.113503   8.911408   8.659332
ENSMUSG00000020184  13.666191  12.662234  13.831565  13.577504  13.216128
ENSMUSG00000020185   7.094639   6.201575   6.976094   7.303302   7.503461
ENSMUSG00000020189  14.430417  14.520433  14.341310  13.879670  12.958598
ENSMUSG00000020196  11.925505  12.544567  13.255979  12.694107  12.984279
ENSMUSG00000020205  14.105404  15.028714  16.579133  15.765152  15.502654
ENSMUSG00000020211  11.668456  10.244873  10.913213  11.130006  12.

ENSMUSG00000020472  12.322495  12.560597  12.546881  12.787087  13.110383
ENSMUSG00000020474  10.799903   9.854035  10.615649  10.397225  10.447224
ENSMUSG00000020475   7.800549   7.032900   7.242348   7.311955   7.091925
ENSMUSG00000020477  13.968838  12.146957  12.821795  13.278930  12.976683
ENSMUSG00000020483  17.244049  17.868451  16.319801  16.733683  18.008277
ENSMUSG00000020485  11.763772  10.895987  10.659188  12.084719  11.604061
ENSMUSG00000020486  11.503759  10.327475   9.687673  10.532055  11.047335
ENSMUSG00000020492   7.932077   8.587381   8.529679   8.420691   8.751059
ENSMUSG00000020495  10.988126  10.605158  11.157559  11.136780  11.169117
ENSMUSG00000020514  12.010635  11.063336  11.090890  11.391159  11.351966
ENSMUSG00000020515  14.288433  13.414371  13.762351  13.834011  13.363091
ENSMUSG00000020516  13.187874  14.146772  13.710073  13.197937  12.865145
ENSMUSG00000020519  11.858462  10.203144  11.227017  11.207662  10.944243
ENSMUSG00000020520  10.617671  10.0914

ENSMUSG00000020792  13.814111  12.656762  13.413174  13.722632  13.706301
ENSMUSG00000020793   6.143770   7.026064   6.186205   7.499449   7.309492
ENSMUSG00000020794  14.338982  15.587354  15.264201  14.759540  14.110937
ENSMUSG00000020801   9.003052   9.448502   8.519393   9.381726   9.326870
ENSMUSG00000020802  11.574456  10.390195  11.394650  10.859168  11.446404
ENSMUSG00000020804   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000020805   7.793590   8.576106   9.104419   8.148965   8.661275
ENSMUSG00000020810  10.013048   9.642830   9.092012   9.820015  10.475642
ENSMUSG00000020821  17.382409  16.710764  16.995705  17.377834  17.383648
ENSMUSG00000020826   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000020827  14.887330  13.498711  14.308196  14.580206  14.999965
ENSMUSG00000020828  10.821878   9.783837  10.721060  11.130006  10.583070
ENSMUSG00000020829  14.128443  13.145175  13.493033  14.021910  14.095040
ENSMUSG00000020832  12.209698  11.4794

ENSMUSG00000021124  13.601447  13.392011  13.376325  13.714456  13.472048
ENSMUSG00000021125   7.783766   7.544233   7.900697   7.172908   7.477110
ENSMUSG00000021127  17.363502  16.530488  15.821983  16.410152  16.809384
ENSMUSG00000021130   7.248162   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000021134  17.138391  18.679868  17.390952  17.430889  17.855078
ENSMUSG00000021135  18.827354  19.478234  19.350414  19.201552  18.953531
ENSMUSG00000021144  13.044167  11.867930  13.119096  13.241092  12.834120
ENSMUSG00000021148   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000021171  13.699403  14.451174  14.107058  14.010669  13.392779
ENSMUSG00000021177   9.138188   9.354715   9.121664   8.607886   9.383134
ENSMUSG00000021178  15.793198  15.613425  15.407718  15.705370  15.703908
ENSMUSG00000021180   8.895206   9.382859   9.263128   9.053071   8.619988
ENSMUSG00000021182   9.101861   9.004080   8.998602   8.096757   9.287373
ENSMUSG00000021186   9.249823   9.7093

ENSMUSG00000021492  18.712843  17.949671  18.482826  18.570520  18.562721
ENSMUSG00000021493   9.761934   8.816691   9.964925   9.720471  10.005063
ENSMUSG00000021494  12.935929  11.763668  12.987366  13.214731  13.070965
ENSMUSG00000021499   6.143770   6.201575   6.989345   7.311955   6.937454
ENSMUSG00000021500  14.428975  14.995474  14.862710  14.639215  14.229260
ENSMUSG00000021504  11.427504  10.142839  11.227363  11.030126  11.098369
ENSMUSG00000021510  10.925309  11.879145  10.584736  10.515976  10.152363
ENSMUSG00000021518  14.274145  14.634480  14.137949  14.283017  14.545297
ENSMUSG00000021519  12.235257  12.546864  12.657928  12.046944  12.356673
ENSMUSG00000021520  15.830162  15.704127  15.382903  15.251157  15.214479
ENSMUSG00000021536   6.869273   6.201575   7.242348   7.045636   7.345713
ENSMUSG00000021540  13.667446  14.279998  13.720728  13.993939  14.001612
ENSMUSG00000021541   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000021546  16.412607  16.9881

...

ENSMUSG00000020734   7.047232   6.854609   7.304521   6.958372   6.114233
ENSMUSG00000020736  12.101687  12.315429  11.462656  11.558295  11.370427
ENSMUSG00000020738  11.722185  11.921167  11.610460  12.379841  12.031717
ENSMUSG00000020739  11.688246  11.833826  11.674169  11.366276  11.894684
ENSMUSG00000020740  11.976644  11.882637  11.767212  12.518051  11.022354
ENSMUSG00000020744  12.145887  13.026494  12.917523  12.078412  13.443118
ENSMUSG00000020745  15.386032  15.379438  15.607646  14.631292  15.599111
ENSMUSG00000020758   9.417906   8.159264   8.966016   9.262414   7.646604
ENSMUSG00000020766  13.348782  12.476762  12.514658  13.079162  14.228429
ENSMUSG00000020774   9.330505   9.732132   9.631584   9.794425  12.609512
ENSMUSG00000020775  14.893962  14.806110  13.203796  14.597819  13.837419
ENSMUSG00000020776  12.339168  11.381528  11.130839  11.412772  10.987958
ENSMUSG00000020777  19.327474  18.980703  19.416341  19.681413  20.170208
ENSMUSG00000020778   8.837801   8.6063

ENSMUSG00000021065   9.908948   9.758640   9.866555   9.842100   9.629451
ENSMUSG00000021067  12.410128  12.630840  12.425693  12.716772  11.812906
ENSMUSG00000021069  15.477641  15.203314  15.262887  15.622818  18.052203
ENSMUSG00000021070   8.124982   8.118124   8.486854   8.425647   7.245432
ENSMUSG00000021071   6.760215   6.854609   6.677690   6.104630   6.114233
ENSMUSG00000021076  13.377263  13.608413  13.285991  12.365373  14.005080
ENSMUSG00000021095   6.110869   6.138584   6.076161   6.104630   6.114233
ENSMUSG00000021096  16.001600  16.176721  15.690910  15.403156  16.071254
ENSMUSG00000021102  14.744140  14.677941  13.267983  13.728086  13.761750
ENSMUSG00000021103  10.682540  11.678098  11.097870  10.178877  10.840649
ENSMUSG00000021108  10.695539  10.414391  10.413107  10.244554   9.156802
ENSMUSG00000021109  14.255931  14.412430  14.558046  14.025379  14.472318
ENSMUSG00000021111  16.140378  16.236755  15.999332  16.036273  15.369892
ENSMUSG00000021113  10.673908  11.1246

ENSMUSG00000021431  12.382674  12.368860  12.432888  12.467843  13.773279
ENSMUSG00000021432  12.471293  12.163100  11.770173  11.630074  12.259146
ENSMUSG00000021448   6.980148   6.138584   6.384480   6.104630   6.114233
ENSMUSG00000021451  10.596868  10.257371  10.638718  11.819683   8.757236
ENSMUSG00000021456   9.368030   8.890013   9.851846   9.147616   7.109080
ENSMUSG00000021457  12.880556  12.247009  12.746309  13.330705   9.598528
ENSMUSG00000021458  13.379052  13.109338  13.679109  12.665443  14.070142
ENSMUSG00000021460  13.650820  13.310634  12.450710  12.635778  13.988100
ENSMUSG00000021461   9.479320   9.458250   9.755947   9.831160  10.224015
ENSMUSG00000021466  10.685481  10.938435  10.941926  11.473304  12.618600
ENSMUSG00000021468  13.640794  13.957344  13.379708  12.878595  13.592529
ENSMUSG00000021469   6.973558   7.076616   6.858586   6.104630   6.114233
ENSMUSG00000021476  13.685455  13.674545  13.281807  13.354917  14.356225
ENSMUSG00000021477  19.222364  18.3480

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   6.113322   6.121520
ENSMUSG00000021484  16.270871  16.563128  16.561961  16.210157
ENSMUSG00000021486  16.714461  16.037879  15.835459  16.681894
ENSMUSG00000021488  14.517779  15.190538  14.924563  14.627111
ENSMUSG00000021490   6.121568   6.159537   6.113322   6.121520
ENSMUSG00000021492  17.015414  17.696143  17.789765  16.956717
ENSMUSG00000021493  12.039436   9.232546   9.278204  11.997205
ENSMUSG00000021494  12.824619  12.180486  12.330442  12.713195
ENSMUSG00000021499   7.163515   7.145651   7.131066   7.017264
ENSMUSG00000021500  14.335599  14.919751  14.977591  14.420111
ENSMUSG00000021504  10.467020  10.358933  10.190286  10.707597
ENSMUSG00000021510  11.470823  11.804250  11.376588  11.060616
ENSMUSG00000021518  13.269132  14.123766  13.651003  13.275735
ENSMUSG00000021519  12.023526  12.782890  12.397863  12.023891
ENSMUSG00000021520  14.143609  15.564893  15.199766  14.116477
ENSMUSG00000021536   7.109850   7.553096   7.235322   7.274671
ENSMUSG00000021540  13.616567  1

In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12
[13,]        13
[14,]        14
[15,]        15
[16,]        16
[17,]        17
[18,]        18
[19,]        19
[20,]        20
[21,]        21
[22,]        22
[23,]        23
[24,]        24
[25,]        25
[26,]        26
[27,]        27
[28,]        28
[29,]        29
[30,]        30
[31,]        31
[32,]        32
[33,]        33
[34,]        34
[35,]        35
[36,]        36
[37,]        37
[38,]        38
[39,]        39
[40,]        40
[41,]        41
[42,]        42
[43,]        43
[44,]        44
[45,]        45
[46,]        46
[47,]        47
[48,]        48
[49,]        49
[50,]        50
[51,]        51
[52,]        52
[53,]        53
[54,]        54


In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817614 SRR1636589 SRR7817660 SRR7817635 SRR3593524
ENSMUSG00000000001  15.766115  16.206252  15.903869  15.643068  15.187207
ENSMUSG00000000028   8.192656   8.033180   8.178474   8.227609   8.052549
ENSMUSG00000000049  20.428537  20.488350  20.107555  20.364601  20.511828
ENSMUSG00000000058   9.074042   9.461244   8.880225   9.012768   8.884885
ENSMUSG00000000085  11.080172  10.271753  10.981584  10.914202  11.019693
ENSMUSG00000000088  16.883814  15.965222  16.253726  16.296999  15.965910
ENSMUSG00000000120  10.454235   8.157402   8.585624   9.345581   9.341242
ENSMUSG00000000126   7.573521   7.630841   8.057393   9.015819   7.832096
ENSMUSG00000000127  10.357501  10.580490  10.081896  10.180350   9.926174
ENSMUSG00000000142  10.417003  10.841197  10.406504  10.712733  12.941788
ENSMUSG00000000149  16.137205  15.249635  16.222956  16.745759  15.309276
ENSMUSG00000000154  15.712191  15.011718  15.597167  15.783425  15.432183
ENSMUSG00000000168  15.178987  15.0579

ENSMUSG00000001105  13.580379  13.038391  13.508788  13.183905  13.243865
ENSMUSG00000001119   9.944380  10.522708   9.127380   9.628098  10.692607
ENSMUSG00000001123  18.580032  18.336787  17.643004  18.428865  18.605928
ENSMUSG00000001127  15.433419  14.927007  15.365722  15.359634  15.351480
ENSMUSG00000001128  11.908512  10.823286  11.566277  10.659824  12.742939
ENSMUSG00000001131   6.143770   6.935375   6.989345   6.206065   6.963721
ENSMUSG00000001143  12.183546  11.362940  11.364368  11.404879  12.448329
ENSMUSG00000001155  18.969655  18.195859  18.717635  18.830277  18.327635
ENSMUSG00000001158  11.499948  11.514025  11.502948  11.688330  12.043915
ENSMUSG00000001173  10.684809  11.141026  10.603768  11.003529  10.816226
ENSMUSG00000001175  17.456925  16.492520  17.253846  17.125305  16.738160
ENSMUSG00000001211  17.066774  16.389797  16.989731  16.972434  17.270901
ENSMUSG00000001225   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000001228   7.935750   7.4812

ENSMUSG00000002249  11.555675  10.383664  10.649238  11.423055  11.393948
ENSMUSG00000002250  13.991809  10.171987  10.105505  10.011591  12.192060
ENSMUSG00000002257   7.440113   7.032900   7.556444   7.038724   7.900899
ENSMUSG00000002279  13.763022  13.002081  13.626589  13.852474  13.543833
ENSMUSG00000002289  14.104387  15.491313  16.106665  15.286232  15.418798
ENSMUSG00000002297   8.684529   8.699519   8.871875   9.207647   8.003670
ENSMUSG00000002307  11.673004  11.123875  11.389466  12.051333  12.418359
ENSMUSG00000002308   9.418530   9.307558   9.442087   9.115916  10.055462
ENSMUSG00000002324   9.255069   8.096864   8.691994   8.999489   8.781178
ENSMUSG00000002325  13.781564  13.542556  13.003679  13.542573  14.251930
ENSMUSG00000002326  11.636583  11.493601  11.339347  11.844312  12.118670
ENSMUSG00000002346  16.016712  15.412737  15.997836  15.825714  16.322713
ENSMUSG00000002365  14.221536  14.184004  14.515766  14.862384  14.060806
ENSMUSG00000002379  14.234675  13.2952

ENSMUSG00000003379   7.094639   7.870801   6.989345   8.339006   8.162649
ENSMUSG00000003402  16.181949  15.647637  15.922860  15.860199  16.485637
ENSMUSG00000003411   7.375106   7.032900   7.540508   7.276626   7.383146
ENSMUSG00000003418   7.094639   7.032900   6.989345   6.206065   6.177154
ENSMUSG00000003421  13.008282  11.517179  11.884908  12.366163  12.229644
ENSMUSG00000003429  14.470884  15.230463  13.132136  13.403102  14.608624
ENSMUSG00000003435  14.865886  14.250080  14.436392  14.655179  15.370697
ENSMUSG00000003436   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000003437  13.547985  13.092871  13.416067  13.170147  13.709524
ENSMUSG00000003444   9.755638   8.248660   9.468701   8.842520   9.544501
ENSMUSG00000003452   6.143770   7.032900   6.186205   7.045636   7.370890
ENSMUSG00000003458  13.873785  14.399501  14.273980  14.314701  14.555811
ENSMUSG00000003464  15.982246  16.254150  16.290655  16.094125  16.680560
ENSMUSG00000003476   7.375106   7.6028

ENSMUSG00000004654   6.143770   6.763517   7.419324   6.206065   6.916709
ENSMUSG00000004655  14.981367  14.937458  13.529139  14.775603  15.807269
ENSMUSG00000004665   9.877352   8.918412   8.634348   9.110106   9.199584
ENSMUSG00000004667  13.159630  11.771580  12.818381  12.746098  12.697286
ENSMUSG00000004668   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000004677  13.361081  12.341343  13.624591  13.790466  13.090982
ENSMUSG00000004709   7.094639   6.201575   6.989345   7.311955   7.521411
ENSMUSG00000004730  10.407287  10.297230   9.972258  10.149590  11.207797
ENSMUSG00000004768  11.050450  12.209544  11.306709  11.296924  10.133863
ENSMUSG00000004771  14.780256  14.704586  14.783005  14.762227  14.347889
ENSMUSG00000004788  12.339992  11.530801  11.683092  12.306386  11.973195
ENSMUSG00000004789  17.153714  16.913264  17.171441  17.078993  16.824800
ENSMUSG00000004791   6.143770   6.201575   6.186205   6.206065   6.177154
ENSMUSG00000004815  14.225819  13.6847

...

ENSMUSG00000009995  13.092827  13.165594  13.216520  13.246059
ENSMUSG00000010021   7.606103   6.913871   7.237393   7.736720
ENSMUSG00000010025  17.316362  17.462821  17.522100  17.262676
ENSMUSG00000010045  12.649247  12.522336  11.965309  12.543838
ENSMUSG00000010047  13.380839  12.941814  13.160718  13.536767
ENSMUSG00000010051  10.955569  14.102897  14.877203  11.153161
ENSMUSG00000010057  11.304139  11.050268  10.803437  11.219319
ENSMUSG00000010064  17.705697  19.362130  19.382010  17.686569
ENSMUSG00000010066   6.121568   6.159537   6.113322   6.121520
ENSMUSG00000010095  17.755072  14.966362  15.942872  17.794751
ENSMUSG00000010097  15.057910  14.925363  14.548376  14.944816
ENSMUSG00000010110  13.930838  14.655330  14.938045  13.797584
ENSMUSG00000010122   9.457115  15.950744  15.514623   9.497572
ENSMUSG00000010142   9.141670   7.651852   7.686029   9.053960
ENSMUSG00000010311   6.121568   6.159537   6.783219   6.121520
ENSMUSG00000010376  14.504923  13.984192  13.740343  14

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   6.159537   6.113322   6.121520
ENSMUSG00000020805   6.121568   7.145651   8.262811   6.121520
ENSMUSG00000020810  11.091160   9.909964   9.854375  11.348446
ENSMUSG00000020821  17.479164  16.232461  16.193227  17.387290
ENSMUSG00000020826  10.497933   6.159537   7.486150  10.170524
ENSMUSG00000020827  14.053761  13.872251  13.662232  13.936115
ENSMUSG00000020828  10.341741   9.917262   9.749711  10.623970
ENSMUSG00000020829  12.394878  12.924630  13.266793  12.316107
ENSMUSG00000020832  11.761597  11.406851  11.387899  11.800405
ENSMUSG00000020838   7.137341   7.777886   7.625475   6.978548
ENSMUSG00000020840  13.725515  14.811857  14.990449  13.761724
ENSMUSG00000020841  13.762397  13.985274  14.236420  13.689778
ENSMUSG00000020849  16.298022  16.805196  16.655482  16.279406
ENSMUSG00000020850  16.070925  16.391700  16.512085  16.030814
ENSMUSG00000020857  14.555288  14.776180  14.845930  14.475522
ENSMUSG00000020859  15.734975  15.894853  15.922039  15.682956
ENSMUSG00000020863  1

ENSMUSG00000021216   6.121568   6.159537   6.988702   6.121520
ENSMUSG00000021218  17.393610  17.884208  17.899293  17.356922
ENSMUSG00000021219   7.145324   7.555183   7.975954   7.450708
ENSMUSG00000021222   9.158049   9.824903   9.733799   8.957487
ENSMUSG00000021224  13.061197  13.894363  14.085791  13.062266
ENSMUSG00000021226  15.262891  10.926403  11.785563  15.320285
ENSMUSG00000021228  14.030232  16.832777  16.556498  13.813815
ENSMUSG00000021236  19.545115  18.529141  18.498290  19.525527
ENSMUSG00000021238  16.568357  18.693284  18.583360  16.477308
ENSMUSG00000021240  10.367016  11.175935  10.932142  10.814191
ENSMUSG00000021241  12.123150  12.635925  12.259480  12.164033
ENSMUSG00000021242  15.421754  15.582977  15.477146  15.449678
ENSMUSG00000021243  11.410849  11.856618  11.598542  11.329386
ENSMUSG00000021245  11.868729  11.748745  11.712334  12.309637
ENSMUSG00000021248  14.440331  14.539352  14.612243  14.365365
ENSMUSG00000021250  10.379957   8.355308   9.137038  10

ENSMUSG00000021635  11.956706  11.826923  11.603496  12.113135
ENSMUSG00000021638  10.997071  11.944831  13.211878  10.488522
ENSMUSG00000021639  10.912953  11.370415  11.448489  11.142978
 [ reached 'max' / getOption("max.print") -- omitted 7446 rows ]


In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_40.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
[1] 13 18 34 47 50

$Fold02
[1]  8 17 30 37 49

$Fold03
[1] 10 21 23 39 40

$Fold04
[1]  1  9 11 35 43 44

$Fold05
[1]  7 14 22 32 48

$Fold06
[1] 15 16 24 41 52 54

$Fold07
[1]  2  6 19 28 36 42

$Fold08
[1] 26 27 45 51 53

$Fold09
[1]  4 20 25 29 31 46

$Fold10
[1]  3  5 12 33 38



In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1]  0 30


In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_40.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_40.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_40.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951175 371.3   13077520 698.5 11228297 599.7
Vcells 14532345 110.9   25842636 197.2 25842636 197.2


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_40.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_40.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1] 13 18 34 47 50
[1] "----------"
[1]  8 17 30 37 49
[1] "----------"
[1] 10 21 23 39 40
[1] "----------"
[1]  1  9 11 35 43 44
[1] "----------"
[1]  7 14 22 32 48
[1] "----------"
[1] 15 16 24 41 52 54
[1] "----------"
[1]  2  6 19 28 36 42
[1] "----------"
[1] 26 27 45 51 53
[1] "----------"
[1]  4 20 25 29 31 46
[1] "----------"
[1]  3  5 12 33 38
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  1 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.849099099099099 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.109696234696235 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  -0.0312774312774313 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  1 : Las

[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.916666666666667 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.772176022176022 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  1 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  1 : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  1 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  1 : Last ECOUNT =  72 

[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  1 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  0.787941787941788 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  1 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89 % remaining..."
[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  0.0167994917994918 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9624798 : Last ARI =  0.25504158004158 : Last ECOU

[1] "Last RXN_ID =  R-MMU-9013004 : Last ARI =  1 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818576 : Last ARI =  0.187052437052437 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029273 : Last ARI =  0.849099099099099 : Last ECOUNT =  71 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8850356 : Last ARI =  0.637040887040887 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1363303 : Last ARI =  0.83974358974359 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-188386 : Last ARI =  0.833333333333333 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-3

[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  0.916666666666667 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =  0.595634095634096 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848484 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9022185 : Last ARI =  0.855509355509356 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2032396 : Last ARI =  0.630630630630631 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-182990 : Last A

...

[1] "Last RXN_ID =  R-MMU-75090 : Last ARI =  0.450854700854701 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77342 : Last ARI =  0.278632478632479 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-449937 : Last ARI =  -0.115384615384615 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9718020 : Last ARI =  0.120252945252945 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6806966 : Last ARI =  0.616666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-140783 : Last ARI =  0.83974358974359 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.2 % remaining..."
[1] "Last RXN_ID

[1] "Last RXN_ID =  R-MMU-9757010 : Last ARI =  0.916666666666667 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9653592 : Last ARI =  0.115552090552091 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9668464 : Last ARI =  0.0481635481635482 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2454118 : Last ARI =  0.36460498960499 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-159425 : Last ARI =  0.916666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-6785607 : Last ARI =  0.807692307692308 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.2 % remaining..."
[1] "Last R

[1] "Last RXN_ID =  R-MMU-975633 : Last ARI =  0.599099099099099 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9027532 : Last ARI =  1 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-917888 : Last ARI =  0.357264957264957 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6800315 : Last ARI =  0.698198198198198 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-200424 : Last ARI =  0.093029568029568 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-425822 : Last ARI =  0.0899110649110649 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-884

[1] "Last RXN_ID =  R-MMU-8879117 : Last ARI =  0.713963963963964 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139909 : Last ARI =  -0.077015477015477 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8851979 : Last ARI =  0.402685377685378 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9704248 : Last ARI =  0.518451143451143 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5629230 : Last ARI =  0.916666666666667 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-1482961 : Last ARI =  0.141146916146916 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7 % remaining..."
[1] "Last RXN_ID =

[1] "Last RXN_ID =  R-MMU-1855172 : Last ARI =  0.270299145299145 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807055 : Last ARI =  0.682432432432432 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-1462039 : Last ARI =  0.586018711018711 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-3341294 : Last ARI =  0.199006699006699 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-8952128 : Last ARI =  0.716666666666667 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2 % remaining..."
[1] "Last RXN_ID =  R-MMU-9606623 : Last ARI =  0.864864864864865 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  1.9 % remaining..."
[1] "Last RXN_ID =

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_40.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_40.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_40.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_40.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  9.166667e-01     14
R-MMU-170666.30.00   R-MMU-170671        0  9.166667e-01     14
R-MMU-170671.0.00    R-MMU-170674        0  9.166667e-01     14
R-MMU-170671.30.00   R-MMU-170686        0  9.166667e-01     14
R-MMU-170674.0.00    R-MMU-380073        0  1.000000e+00    182
R-MMU-170674.30.00   R-MMU-392187        0  1.000000e+00     22
R-MMU-170686.0.00    R-MMU-392195        0  1.000000e+00     22
R-MMU-170686.30.00   R-MMU-392202        0  1.000000e+00     22
R-MMU-380073.0.00    R-MMU-749454        0  1.000000e+00    178
R-MMU-380073.30.00   R-MMU-749456        0  1.000000e+00    178
R-MMU-392187.0.00    R-MMU-751001        0  1.000000e+00     21
R-MMU-392187.30.00  R-MMU-8964229        0  1.000000e+00      5
R-MMU-392195.0.00   R-MMU-8964242        0  1.000000e+00      5
R-MMU-392195.30.00  R-MMU-8964329        0  1.000000e+00      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-205099.30.00  R-MMU-5654224        0  9.166667e-01     12
R-MMU-205112.0.00   R-MMU-5654392        0  1.000000e+00     20
R-MMU-205112.30.00  R-MMU-5654397        0  9.166667e-01     16
R-MMU-205117.0.00   R-MMU-5654399        0  9.166667e-01     16
R-MMU-205117.30.00  R-MMU-5654402        0  1.000000e+00     20
R-MMU-209532.0.00   R-MMU-5654404        0  9.166667e-01     16
R-MMU-209532.30.00  R-MMU-5654406        0  9.166667e-01     18
R-MMU-209566.0.00   R-MMU-5654407        0  9.166667e-01     16
R-MMU-209566.30.00  R-MMU-5654408        0  9.166667e-01     12
R-MMU-9012761.0.00  R-MMU-5654409        0  9.166667e-01     12
R-MMU-9012761.30.00 R-MMU-5654413        0  1.000000e+00     17
R-MMU-3247837.0.00  R-MMU-5654416        0  9.166667e-01     15
R-MMU-3247837.30.00 R-MMU-5654418        0  1.000000e+00     14
R-MMU-3247840.0.00  R-MMU-5654422        0  1.000000e+00     14
R-MMU-3247840.30.00 R-MMU-5654423        0  1.000000e+00     16
R-MMU-3247843.0.00  R-MMU-5654425       

R-MMU-5654224.30.00 R-MMU-9011082        0  6.471760e-01      2
R-MMU-5654392.0.00  R-MMU-9011144        0 -6.491684e-02      3
R-MMU-5654392.30.00 R-MMU-9011146        0 -6.491684e-02      3
R-MMU-5654397.0.00  R-MMU-9753028        0  8.333333e-01      5
R-MMU-5654397.30.00 R-MMU-9753126        0  8.333333e-01      5
R-MMU-5654399.0.00   R-MMU-514604        0  4.734581e-01      2
R-MMU-5654399.30.00  R-MMU-975389        0  7.815315e-01      4
R-MMU-5654402.0.00  R-MMU-1169394        0  1.000000e+00      5
R-MMU-5654402.30.00 R-MMU-1169398        0  5.507623e-01      5
R-MMU-5654404.0.00  R-MMU-1169402        0  7.657658e-01      6
R-MMU-5654404.30.00 R-MMU-1169403        0  4.706341e-01      4
R-MMU-5654406.0.00  R-MMU-1169405        0  6.119196e-01      5
R-MMU-5654406.30.00 R-MMU-1169406        0  9.324324e-01     12
R-MMU-5654407.0.00  R-MMU-1678842        0  1.000000e+00     13
R-MMU-5654407.30.00 R-MMU-1678843        0  5.382017e-01      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-5654659.30.00 R-MMU-2424487        0  9.166667e-01     22
R-MMU-5654662.0.00   R-MMU-388814        0  9.166667e-01      6
R-MMU-5654662.30.00  R-MMU-388829        0  1.000000e+00      9
R-MMU-5654663.0.00   R-MMU-388831        0  1.000000e+00      8
R-MMU-5654663.30.00  R-MMU-388832        0  1.000000e+00      9
R-MMU-5654664.0.00   R-MMU-388833        0  1.000000e+00     17
R-MMU-5654664.30.00  R-MMU-389158        0  1.000000e+00      9
R-MMU-5654667.0.00   R-MMU-389348        0  9.166667e-01      8
R-MMU-5654667.30.00  R-MMU-389350        0  9.166667e-01      8
R-MMU-5654669.0.00   R-MMU-389352        0  9.166667e-01      7
R-MMU-5654669.30.00  R-MMU-389354        0  9.166667e-01      7
R-MMU-5654672.0.00   R-MMU-389381        0  8.490991e-01      6
R-MMU-5654672.30.00  R-MMU-389758        0  1.000000e+00     22
R-MMU-5654673.0.00   R-MMU-389759        0  9.166667e-01      7
R-MMU-5654673.30.00  R-MMU-389762        0  9.166667e-01      5
R-MMU-5654677.0.00   R-MMU-434836       

R-MMU-9011082.30.00 R-MMU-9692139        0  3.065315e-01      2
R-MMU-9011144.0.00  R-MMU-9692145        0  5.049896e-01      2
R-MMU-9011144.30.00 R-MMU-9692156        0  7.666320e-02      4
R-MMU-9011146.0.00  R-MMU-9692157        0  7.666320e-02      4
R-MMU-9011146.30.00 R-MMU-9692268        0  5.129880e-01      2
R-MMU-9753028.0.00   R-MMU-450434        0  2.956803e-02      1
R-MMU-9753028.30.00  R-MMU-450466        0  9.230769e-01     45
R-MMU-9753126.0.00   R-MMU-450551        0  6.888427e-01      7
R-MMU-9753126.30.00  R-MMU-450580        0  9.166667e-01     12
R-MMU-514604.0.00     R-MMU-72103        0  8.555094e-01     43
R-MMU-514604.30.00  R-MMU-9770119        0  9.230769e-01     69
R-MMU-975389.0.00   R-MMU-9770141        0  9.230769e-01    123
R-MMU-975389.30.00  R-MMU-9770142        0  9.230769e-01    132
R-MMU-1169394.0.00  R-MMU-9770145        0  1.000000e+00    145
R-MMU-1169394.30.00 R-MMU-9770236        0  9.230769e-01    119
R-MMU-1169398.0.00  R-MMU-6798748       

...

R-MMU-9699581.0.00   R-MMU-391940        0  3.174752e-01      2
R-MMU-9699581.30.00  R-MMU-428625        0  7.046085e-01      1
R-MMU-5619439.0.00   R-MMU-888592        0  7.046085e-01      1
R-MMU-5619439.30.00 R-MMU-5362564        0  1.000000e+00      2
R-MMU-417896.0.00     R-MMU-71707        0  1.000000e+00      3
R-MMU-417896.30.00   R-MMU-174391        0  2.947794e-01      1
R-MMU-9634937.0.00  R-MMU-2408551        0  2.947794e-01      1
R-MMU-9634937.30.00 R-MMU-8869606        0  1.720085e-01      1
R-MMU-381608.0.00   R-MMU-8869633        0  1.720085e-01      1
R-MMU-381608.30.00   R-MMU-844619        0  3.851640e-01      1
R-MMU-8959510.0.00   R-MMU-877396        0  3.851640e-01      1
R-MMU-8959510.30.00  R-MMU-202703        0  6.212751e-01      7
R-MMU-9034714.0.00   R-MMU-391152        0  7.721760e-01      3
R-MMU-9034714.30.00  R-MMU-391153        0  7.564103e-01      3
R-MMU-9615249.0.00   R-MMU-391157        0  7.815315e-01      3
R-MMU-9615249.30.00  R-MMU-391158       

R-NUL-9622751.0.00  R-MMU-6814374        0  9.166667e-01      2
R-NUL-9622751.30.00 R-MMU-8944262        0  2.854585e-02      1
R-NUL-9622814.0.00  R-MMU-5686304        0  1.000000e+00      2
R-NUL-9622814.30.00  R-MMU-947499        0 -1.740125e-01      1
R-MMU-391937.0.00    R-MMU-893593        0  3.576461e-01      1
R-MMU-391937.30.00   R-MMU-893596        0  3.576461e-01      1
R-MMU-418918.0.00    R-MMU-893616        0  3.576461e-01      1
R-MMU-418918.30.00   R-MMU-159431        0  9.166667e-01      1
R-MMU-1237325.0.00   R-MMU-192312        0  9.166667e-01      1
R-MMU-1237325.30.00  R-MMU-193491        0  9.166667e-01      1
R-MMU-1247668.0.00  R-MMU-6788650        0  6.571957e-02      1
R-MMU-1247668.30.00  R-MMU-373745        0  3.073458e-01      1
R-MMU-9707504.0.00   R-MMU-446208        0 -1.746939e-01      1
R-MMU-9707504.30.00  R-MMU-448741        0  5.184511e-01      2
R-MMU-9707516.0.00    R-MMU-71200        0  7.657658e-01      1
R-MMU-9707516.30.00 R-MMU-8854255       

R-MMU-9749977.0.00  R-MMU-2730862        0 -3.235158e-02      1
R-MMU-9749977.30.00 R-MMU-6799959        0  7.972973e-01      2
R-MMU-266204.0.00   R-MMU-9673053        0  5.090956e-01      1
R-MMU-266204.30.00  R-MMU-9673054        0  5.090956e-01      1
R-MMU-419334.0.00    R-MMU-203156        0  1.000000e+00      1
R-MMU-419334.30.00   R-MMU-192097        0  3.693982e-01      1
R-MMU-5638333.0.00   R-MMU-193789        0  3.693982e-01      1
R-MMU-5638333.30.00  R-MMU-193816        0  3.693982e-01      1
R-MMU-5649800.0.00  R-MMU-5665999        0  1.170016e-01      1
R-MMU-5649800.30.00  R-MMU-389523        0  4.059829e-03      1
R-MMU-5649802.0.00   R-MMU-389919        0  1.819647e-01      2
R-MMU-5649802.30.00 R-MMU-6790089        0  4.059829e-03      1
R-MMU-9696274.0.00  R-MMU-9706304        0  7.337145e-01      2
R-MMU-9696274.30.00 R-MMU-8876283        0  1.027778e-01      1
R-MMU-109380.0.00    R-MMU-975926        0  5.147436e-01      1
R-MMU-109380.30.00  R-MMU-6786295       

R-NUL-2467624.0.00  R-MMU-9704297        0  2.729845e-01      1
R-NUL-2467624.30.00  R-MMU-162797        0 -3.858281e-02      1
R-MMU-9692376.0.00   R-MMU-965067        0  4.870871e-01      1
R-MMU-9692376.30.00  R-NUL-421961        0  6.730769e-02      1
R-MMU-9695917.0.00  R-MMU-9634846        0  3.193520e-01      2
R-MMU-9695917.30.00 R-MMU-9639576        0  5.229845e-01      1
R-MMU-9695949.0.00  R-MMU-9640074        0  5.229845e-01      1
R-MMU-9695949.30.00 R-MMU-9769840        0  7.815315e-01      3
R-MMU-8954056.0.00  R-MMU-8873850        0 -6.440286e-02      1
R-MMU-8954056.30.00  R-MMU-426560        0  1.940402e-02      1
R-MMU-70449.0.00    R-MMU-8953037        0  1.000000e+00      1
R-MMU-70449.30.00    R-MMU-162729        0  1.000000e+00      2
R-MMU-70482.0.00     R-MMU-451056        0  2.581428e-02      1
R-MMU-70482.30.00   R-MMU-2404142        0  4.867752e-02      1
R-MMU-114263.0.00   R-MMU-2404144        0  4.867752e-02      1
R-MMU-114263.30.00  R-MMU-2855259       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_40.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_40.txt",sep=""), header=FALSE)
print(sid)

           V1
1  SRR7817614
2  SRR1636589
3  SRR7817660
4  SRR7817635
5  SRR3593524
6  SRR7817651
7  SRR7817675
8  SRR7817687
9  SRR7817676
10 SRR7817673
11 SRR7817615
12 SRR1636588
13 SRR7817612
14 SRR7817696
15 SRR7817684
16 SRR1636591
17 SRR7817623
18 SRR1636592
19 SRR4317610
20 SRR7817628
21 SRR7817647
22 SRR7817616
23 SRR7817697
24 SRR4317609
25 SRR7817611
26 SRR7817640
27 SRR7817699
28 SRR4317657
29 SRR7817641
30 SRR7817634
31 SRR7817654
32 SRR3593574
33 SRR1636674
34 SRR7817629
35 SRR7817655
36 SRR7817622
37 SRR7817642
38 SRR7817657
39 SRR4317656
40 SRR7817704
41 SRR7817701
42 SRR7817681
43 SRR4317655
44 SRR4317660
45 SRR1636675
46 SRR7817679
47 SRR7817620
48 SRR7817653
49 SRR7817619
50 SRR7817667
51 SRR7817632
52 SRR1636671
53 SRR1636673
54 SRR7817631


In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  
     colors = c("red", "green"),  
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_40.html", selfcontained = TRUE)